In [3]:
import  sys
from  gastrodon  import RemoteEndpoint,QName,ttl,URIRef,inline
import pandas as pd
pd.options.display.width=120
pd.options.display.max_colwidth=100

In [4]:
prefixes=inline("""
    @prefix l: <https://slovník.gov.cz/legislativní/sbírka/111/2009/pojem/> .
    @prefix a: <https://slovník.gov.cz/agendový/104/pojem/> .
""").graph

In [5]:
endpoint=RemoteEndpoint(
    "http://94.199.43.20/odrpp/sparql/"
    ,default_graph=""
    ,prefixes=prefixes
    ,base_uri="http://szrcr.cz/odrpp/zdroj/"
)

Následující dotaz vrátí seznam orgánů veřejné moci evidovaných v RPP. Výsledek je omezen na 10 řádků, což lze upravit v klauzili LIMIT.

In [6]:
ovm=endpoint.select("""
PREFIX l: <https://slovník.gov.cz/legislativní/sbírka/111/2009/pojem/>
PREFIX a: <https://slovník.gov.cz/agendový/104/pojem/>

SELECT ?ovm
WHERE {
  ?ovm a l:orgán-veřejné-moci .
}
LIMIT 10
""")
ovm

,ovm
0,<orgán-veřejné-moci/00006572>
1,<orgán-veřejné-moci/00025402>
2,<orgán-veřejné-moci/00025798>
3,<orgán-veřejné-moci/04095316>
4,<orgán-veřejné-moci/05108861>
5,<orgán-veřejné-moci/05553521>
6,<orgán-veřejné-moci/05575389>
7,<orgán-veřejné-moci/05800226>
8,<orgán-veřejné-moci/06216871>
9,<orgán-veřejné-moci/48510190>


Ve výsledku vidíme pouze IRI orgánů veřejné moci. Pokud chceme získat i hodnoty jejich vlastností je nutné znát IRI těchto vlastností. To lze zjistit v dokumentaci https://data.gov.cz/otevřené-formální-normy/registr-práv-a-povinností/orgány-veřejné-moci. Pokud se nám nechce dokumentaci číst, můžeme využít následující dotaz, který vybere všechny vlastnosti všech orgánů veřejné moci a vrátí jejich množinu (tj. odstraní duplicity).

In [7]:
vlastnosti_ovm=endpoint.select("""
PREFIX l: <https://slovník.gov.cz/legislativní/sbírka/111/2009/pojem/>
PREFIX a: <https://slovník.gov.cz/agendový/104/pojem/>
SELECT DISTINCT ?vlastnost
WHERE {
  ?ovm  a l:orgán-veřejné-moci ;
    ?vlastnost [] .
}
ORDER BY ?vlastnost
""")
vlastnosti_ovm

,vlastnost
0,rdf:type
1,a:má-datum-ukončení-výkonu-působnosti-orgánu-veřejné-moci
2,l:je-vnitřní-organizační-jednotkou
3,l:má-adresu-sídla-orgánu-veřejné-moci
4,l:má-datovou-schránku-orgánu-veřejné-moci
5,l:má-datum-zahájení-výkonu-působnosti-orgánu-veřejné-moci
6,l:má-identifikační-číslo-osoby-orgánu-veřejné-moci
7,l:má-identifikátor-orgánu-veřejné-moci
8,l:má-název-orgánu-veřejné-moci
9,l:má-pracoviště-vykonávající-úkon


Získaný seznam vlastností orgánů veřejné moci můžeme použít pro rozšíření vlastností, které si pro jednotlivé orgány veřejné moci vracíme, např. můžeme vracet jejich názvy jako v následujícím dotazu.

In [8]:
ovm=endpoint.select("""
PREFIX l: <https://slovník.gov.cz/legislativní/sbírka/111/2009/pojem/>
PREFIX a: <https://slovník.gov.cz/agendový/104/pojem/>
SELECT *
WHERE {
  ?ovm  a l:orgán-veřejné-moci ;
    l:má-název-orgánu-veřejné-moci ?ovmNazev .
}
LIMIT 10
""")
ovm

,ovm,ovmNazev
0,<orgán-veřejné-moci/00006572>,Česká republika - Kancelář Poslanecké sněmovny
1,<orgán-veřejné-moci/00025402>,Úřad pro mezinárodněprávní ochranu dětí
2,<orgán-veřejné-moci/00025798>,Česká geologická služba
3,<orgán-veřejné-moci/04095316>,Centrum pro regionální rozvoj České republiky
4,<orgán-veřejné-moci/05108861>,Agentura pro podnikání a inovace
5,<orgán-veřejné-moci/05553521>,Úřad pro přístup k dopravní infrastruktuře
6,<orgán-veřejné-moci/05575389>,Finanční analytický úřad
7,<orgán-veřejné-moci/05800226>,Národní úřad pro kybernetickou a informační bezpečnost
8,<orgán-veřejné-moci/06216871>,Vojenská policie
9,<orgán-veřejné-moci/48510190>,Nejvyšší soud


Výstup dále rozšíříme o IČ.

In [9]:
ovm=endpoint.select("""
PREFIX l: <https://slovník.gov.cz/legislativní/sbírka/111/2009/pojem/>
PREFIX a: <https://slovník.gov.cz/agendový/104/pojem/>
SELECT *
WHERE {
  ?ovm  a l:orgán-veřejné-moci ;
    l:má-název-orgánu-veřejné-moci ?ovmNazev ;
    l:má-identifikační-číslo-osoby-orgánu-veřejné-moci ?ovmIC .
}
LIMIT 10
""")
ovm

,ovm,ovmNazev,ovmIC
0,<orgán-veřejné-moci/00006572>,Česká republika - Kancelář Poslanecké sněmovny,6572
1,<orgán-veřejné-moci/00025402>,Úřad pro mezinárodněprávní ochranu dětí,25402
2,<orgán-veřejné-moci/00025798>,Česká geologická služba,25798
3,<orgán-veřejné-moci/04095316>,Centrum pro regionální rozvoj České republiky,4095316
4,<orgán-veřejné-moci/05108861>,Agentura pro podnikání a inovace,5108861
5,<orgán-veřejné-moci/05553521>,Úřad pro přístup k dopravní infrastruktuře,5553521
6,<orgán-veřejné-moci/05575389>,Finanční analytický úřad,5575389
7,<orgán-veřejné-moci/05800226>,Národní úřad pro kybernetickou a informační bezpečnost,5800226
8,<orgán-veřejné-moci/48510190>,Nejvyšší soud,48510190
9,<orgán-veřejné-moci/65993390>,Ředitelství silnic a dálnic ČR,65993390
